# Grundlagen der Multimediatechnik
## Wintersemester 2021/2022

## Übungslatt 3
<b>Wichtig</b>: Bitte kennzeichnen Sie Ihre Abgabe mit Ihrem Namen und dem Namen Ihres Übungspartners. <br>
Laden Sie bitte Ihre Abgabe als PDF-/ZIP-Datei auf der ILIAS-Plattform hoch. Andere Dateiformate sowie Scans von handgeschriebenen Abgaben werden nicht gewertet. Achten Sie darauf, nur kompilierbaren, kommentierten Code abzugeben. Nicht-kompilierbarer Code wird mit <b>0 Punkten</b> bewertet! \\
Alle Abgaben müussen folgender Namenskonvention entsprechen:  `gmt_uebungXX_nachname1_nachname2.format` <br>
Im Falle Ihres Notebooks: `gmt_uebungXX_nachname1_nachmame2.ipynb`

### Aufgabe 2 - Lauflängenkodierung (8 Punkte)

1. Was versteht man unter Entropiekodierung?
2. Erklären Sie kurz, wie die in der Vorlesung besprochene Lauflängenkodierung funktioniert und geben Sie ein Beispiel, bei dem die Lauflängenkodierung Anwendung findet.

1) Die Entropiekodierung nutzt die Redundanz der Quelle (z.B ein Wort), um kurze Codewoerter zu finden
--> verlustfreie Kompression und Datensemantik wird beachtet darunter zaehlen: RLE und LZW

2) RLE ist gut bei haeufigen Abfolgen an identischen Symbolen bspw.
$Q = wwwwwwwwwBwwwwwwBBBBwwwwww,\ |Q|=26$
$\\$ wir sehen, dass es einiges an Redundanz gibt. Jetzt ist die Idee von RLE, dass wir die Folge
an gleichen Zeichen kodieren: $[Laenge\ der\ Folge|Zeichen]$ d.h
$Q' = RLE(Q) = 9w1B6w4B6w, |Q'|=10$
$\\$ es muessen also 16 Symbole weniger uebertragen werden.
$\\$ Anwendung findet sie z.B beim Uebertragen von Schwarz-Weiss-Seiten oder in Kombination
mit einer modifizierten Huffman-Kodierung bei Fax-Uebertragung zum Einsatz.

3. Vervollständigen Sie die `rle(word)`-Funktion, die einen Eingabestring erhalten soll und den entsprechenden String mittels Lauflängenkodierung kodieren und zurückgeben.

<b>Anmerkung:</b>  Für die Vervollständigung des Skriptes sind keine externen Bibliotheken notwendig und sind somit für die Bearbeitung dieser Aufgabe unzulässig (<b>keine</b> imports!).

In [272]:
def rle(word):
    # Init
    countWithChar = [ 1 , word[0] ]             # structure: [countOfChar, Char]
    output = ''                                 # return string
    endOfWordIndex = len(word)-1

    """
    count occurrence of the same char
    --> when encountering new char or end of word:
    1. save occurrence and counted char
    2. then start with the new char or end if end of word
    (added: removing count of 1 when just char is counted)
    """
    for i in range(1, len(word) ):

        itsTheSameChar = word[i] == countWithChar[1]

        if itsTheSameChar:
            countWithChar[0] += 1
        if not itsTheSameChar \
                or i == endOfWordIndex:

            countOfChar = countWithChar[0]
            char = countWithChar[1]

            if countOfChar == 1:                      # remove 1 when just one char counted
                listToString = char
            else:
                listToString = str(countOfChar)+char  # convert list elements to one string

            output += listToString             # created string gets appended to output string
            countWithChar = [ 1 , word[i] ]    # set to new char found with count=1

    return output

In [273]:
word1 = "lllloooreeemmmiiiiipsuuuuumdddoooollorrr"
word2 = "aaaalliiquamlllucctussssss"

print(rle(word1))
print(rle(word2))

4l3or3e3m5ips5um3d4o2lo3r
4a2l2iquam3lu2ctu6s


### Aufgabe 3 - Burrows-Wheeler-Transformation (11 Punkte)
Ziel dieser Aufgabe ist es, die aus der Vorlesung bekannte Burrows-Wheeler-Transformation zu implementieren.

<b>Anmerkung:</b>  Für die Vervollständigung des Skriptes sind keine externen Bibliotheken notwendig und sind somit für die Bearbeitung dieser Aufgabe unzulässig (<b>keine</b> imports!).

1. Im ersten Schritt müssen Sie die `create_permutation_matrix(word)`-Funktion vervollständigen. Diese bekommt einen String als Input und soll die zugehörige Permutationsmatrix erstellen und lexikographisch sortieren. Anschließend soll die sortierte Matrix zurückgegeben werden.

In [274]:
def create_permutation_matrix(word):

    permMatrix = []      # return permutation matrix
    start = 0            # counter for rotation thru indices

    """
    start creating word with different indices
    start = 0 --> first left rotation (1 step)
    start = 1 --> second ...
    examples: w = ABC -1-> BCA -2-> CAB
    stops when original word is found
    """
    while True:
        permuation = []
        length = len(word)

        for i in range(start, length+start ):
            index = (i+1) % length
            permuation.append( word[index] )

        permuation = ''.join(permuation)
        permMatrix.append(permuation)

        if permuation == word:
            break

        start += 1

    return sorted(permMatrix)

2. Vervollständigen Sie anschließend die `bwt_transform(matrix, word)`-Funktion. Diese bekommt als Parameter die sortierte Matrix aus Schritt 1 sowie das ursprüngliche Wort zur Ermittlung des zugehörigen Index. Am Ende sollen der ermittelte Index sowie die letzte Spalte der Matrix als String zurückgegeben werden.

In [275]:
def bwt_transform(matrix, word):

    output = ''
    index = 0

    for i in range( len(matrix) ):

        matrix[i] = ''.join(matrix[i])
        lastChar = matrix[i][-1]
        output += lastChar

        if matrix[i] == word:
            index = i

    return [output, index]

3. Implementieren Sie anschließend noch die Dekodierung in der `bwt_retransform(encoded_word, idx)`-Funktion. `encoded_word` ist dabei der zurvor kodierte String und `idx` der zugehörige ermittelte Index.

In [276]:
def bwt_retransform(encoded_word, idx):

    # create data struct [pos, charOfEncoding]
    encodedWithPos = []
    for index, char in enumerate(encoded_word):
        encodedWithPos.append([index,char])

    # sort encoded word with saved data abt original pos
    sortedTable = []
    encodedSorted = ''.join(sorted(encoded_word))
    for i in range( len(encodedSorted) ):
        for elem in encodedWithPos:

            if encodedSorted[i] == elem[1]:

                sortedTable.append( elem )
                encodedWithPos.remove( elem )

                break

    index = idx
    decoding = ''

    for elem in sortedTable:

        decoding += sortedTable[index][1]
        index = sortedTable[index][0]

    return decoding

In [277]:
word1 = "bananenbaum"
print("Input: {}" .format(word1))

sorted_matrix1 = create_permutation_matrix(word1)
print("Sortierte Matrix: \n {}" .format(sorted_matrix1))

output1, idx1 = bwt_transform(sorted_matrix1, word1)

print("Output: {} mit Index {}" .format(output1, idx1))
rbwt1 = bwt_retransform(output1, idx1)
print("Rücktransformation: {}" .format(rbwt1))

Input: bananenbaum
Sortierte Matrix: 
 ['ananenbaumb', 'anenbaumban', 'aumbananenb', 'bananenbaum', 'baumbananen', 'enbaumbanan', 'mbananenbau', 'nanenbaumba', 'nbaumbanane', 'nenbaumbana', 'umbananenba']
Output: bnbmnnuaeaa mit Index 3
Rücktransformation: bananenbaum


In [278]:
word2 = "panamamanman"
print("Input: {}" .format(word2))

sorted_matrix2 = create_permutation_matrix(word2)
print("Sortierte Matrix: \n {}" .format(sorted_matrix2))

output2, idx2 = bwt_transform(sorted_matrix2, word2)

print("Output: {} mit Index {}" .format(output2, idx2))
rbwt2 = bwt_retransform(output2, idx2)
print("Rücktransformation: {}" .format(rbwt2))

Input: panamamanman
Sortierte Matrix: 
 ['amamanmanpan', 'amanmanpanam', 'anamamanmanp', 'anmanpanamam', 'anpanamamanm', 'mamanmanpana', 'manmanpanama', 'manpanamaman', 'namamanmanpa', 'nmanpanamama', 'npanamamanma', 'panamamanman']
Output: nmpmmaanaaan mit Index 11
Rücktransformation: panamamanman


<b>Anmerkung:</b> Sollten Probleme bei den ersten beiden Teilaufgaben auftreten, so können Sie zum Testen Ihrer Dekodierungsfunktion folgende Parameter verwenden: `TTROOO,5`

In [279]:
print(bwt_retransform("TTROOO", 5))

TOTORO


4. Bei der Burrows-Wheeler-Transformtion findet keine Kompression statt. Welcher Vorteil ergibt sich dennoch durch die Anwendung der Burrows-Wheeler-Transformation?

Die Sortierung ist allgemein aber auch fuer grosse Datenmengen schnell berechenbar, braucht wenig Speicher
und es existiert kein Informationsverlust.
An sich bietet BWT keine Kompression, jedoch wird in Kombination mit tatsaechlichen Kompressions-Methoden
etwa der RLE oder Hoffman-Kodierung, Kompression moeglich. Diese ist sogar ganz effektiv, da BWT die Quelle
fuer die Methoden gut aufarbeitet.
